In [1]:
import networkx as nx
import pandas as pd

from crosscheck.retrieve import DataRetriever, AUTH_HEADER

In [2]:
query = 'unilateral spatial neglect'
limit = 100

In [3]:
query_processed = query.replace(' ', '+')

In [12]:
search_url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query_processed}&limit={limit}&fields=paperId,referenceCount'


In [13]:
response = DataRetriever._get(search_url)

In [6]:
def build_reference_graph(paper_ids):
    nodes = set()
    edges = set()
    ref_nodes = set()

    for paper_id in paper_ids:
        references_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}/references?fields=paperId,citationCount'
        ref_response = DataRetriever._get(references_url)
        if ref_response.status_code != 200:
            print(ref_response.status_code)
        references = [p['citedPaper']['paperId'] for p in ref_response.json()['data']]

        nodes.add(paper_id)
        for ref_id in references:
            if ref_id is None:
                continue
            nodes.add(ref_id)
            ref_nodes.add(ref_id)
            edges.add((ref_id, paper_id))

    graph = nx.DiGraph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    return graph, ref_nodes

In [15]:
paper_ids = [p['paperId'] for p in response.json()['data']]

KeyError: 'data'

In [16]:
graph, ref_nodes = build_reference_graph(paper_ids)

NameError: name 'paper_ids' is not defined

In [150]:
graph.number_of_edges()

2658

In [151]:
prune_df = pd.DataFrame(graph.out_degree(ref_nodes), 
                        columns=['paperId', 'localCites'])\
             .sort_values(by='localCites', ascending=False)
prune_df

,paperId,localCites
1922,a6d8165549ce808dbef4275e535b9a7faa3ea927,16
1565,57ee1c4e5991bf22f69c2e8ac6ecff2ea21c88e2,10
1021,ae8bcdca8f5a65f82c94a904a9148b62b7364144,8
744,6d15cf8fdae8123533349ea002989ff635b9e9ea,7
282,75c2f465d59739dbc06b70fd34dc3c1b2336103e,7
...,...,...
8,90a32eb8fe61bd619b320eb7726de977a7af20d9,1
7,82b826f8ac079b1f6a3e6ca089125d653e7b04f9,1
6,efde8b637e17fed959b3a7251076b33404798cf7,1
5,d3109cfcc2af3e79391ff4a025309fa26e6bf87d,1
